In [23]:
import pandas as pd
import sqlalchemy
import joblib
import argparse

In [46]:
def postgres_connector(host, port, database, user, password=None):
    user_info = user if password is None else user + ':' + password
    url = 'postgres://%s@%s:%d/%s' % (user_info, host, port, database)
    return sqlalchemy.create_engine(url, client_encoding='utf-8')

def read_data(table):
    query='select * from {}'.format(table)
    df=pd.read_sql(query, engine)
    print('{} table successfully loaded!'.format(table))
    return df

def little_merge(post_table, t2, table_name):
    
    #merge tables
    df_merge=post_table.merge(t2, on='post_key',how='left')
    
    #fill missing value
    df_merge['created_at_hour_y']=df_merge['created_at_hour_y'].fillna(df_merge['created_at_hour_x'])
    df_merge['count']=df_merge['count'].fillna(0)
    
    #exclude data which is not in training domain(10 hr)
    df_merge=df_merge[(df_merge['created_at_hour_y']-\
                       df_merge['created_at_hour_x']).dt.total_seconds()/3600<10]
    df_merge=df_merge[(df_merge['created_at_hour_y']-\
                      df_merge['created_at_hour_x']).dt.total_seconds()/3600>=0]
    df_merge['time_after_create']=(df_merge['created_at_hour_y']\
                                  -df_merge['created_at_hour_x']).dt.total_seconds()/3600
    
    #drop non-training columns
    df=df_merge.drop(['created_at_hour_x','created_at_hour_y'], axis=1)
    
    #features engineering:generate new training features
    df_train=pd.pivot_table(df,values='count',index='post_key',columns='time_after_create')
    df_train=df_train.fillna(0)
    
    #rename the columns
    for i in range(10):
        df_train=df_train.rename(columns={df_train.columns[i]:'{} hour after {}'.format(i,table_name)})
    
    #convert float to int
    #df_train[list(df_train.columns)]=df_train[list(df_train.columns)].astype(int)
    
    print('posts_test table and {} table successfully merge!'.format(table_name))
    return df_train

def post_table_feature_engineer(table):
    
    #adding new training categorical features: created_weekday, created_hour
    table['created_weekday']=table['created_at_hour'].apply(lambda x: x.weekday())
    table['created_time']=table['created_at_hour'].apply(lambda x: x.hour)
    
    #drop non_training features
    table=table.drop(['created_at_hour'], axis=1)
    
    print('posts_test finish feature engineering!')
    return table

def big_merge(t1, t2, t3, t4, post_table):
    
    #outer join the 4 small tables
    df1=t1.merge(t2, on='post_key',how='outer')
    df2=df1.merge(t3, on='post_key',how='outer')
    df3=df2.merge(t4, on='post_key',how='outer')
    
    #left join post_table
    df=post_table.merge(df3, on='post_key',how='left')
    
    #fill missing values
    df=df.fillna(0)
    
    #convert float to int
    df[list(df.columns[1:])]=df[list(df.columns[1:])].astype(int)
    
    # drop non-predicted column
    df=df.drop(['post_key'], axis=1)
    
    print('all table merged successfully!')
    return df

def create_dummy(data, dummy_features):
    
    df=pd.get_dummies(data,columns=dummy_features, drop_first=True)

    print('dummy variable created successfully!')
    return df

def load_model(filepath):
    
    model=joblib.load(filepath)
    
    print('model successfully loaded!')
    return model

def predict_model(model, test_data):
    
    predict=model.predict(test_data)
    print('every post is predicted!')
    return predict

def save_output(data, predict, output_filepath):
    
    post_key=data.post_key
    results = pd.DataFrame({'post_key':post_key, 'is_trending':predict})
    results.to_csv(output_filepath,index=False)
    
    print('output successfully saved!')

In [ ]:
if __name__ == '__main__':
    
    parser = argparse.ArgumentParser(description='Predict if post will become trending post for Dcard posts')
    parser.add_argument('host', help='input database host')
    parser.add_argument('model', help='filepath where model saved')
    parser.add_argument('output', help='filepath where output save')
    args = parser.parse_args()
    
    #access database
    engine = postgres_connector(
    args.host,
    5432,
    "intern_task",
    "candidate",
    "dcard-data-intern-2020"
    )
    
    #load test table
    df_posts=read_data('posts_test')
    df_share=read_data('post_shared_test')
    df_comment=read_data('post_comment_created_test')
    df_like=read_data('post_liked_test')
    df_collect=read_data('post_collected_test')

    #join feature tables and feature engineering
    posts_share=little_merge(df_posts,df_share, 'share')
    posts_comment=little_merge(df_posts,df_comment, 'comment')
    posts_like=little_merge(df_posts,df_like, 'like')
    posts_collect=little_merge(df_posts,df_collect, 'collect')

    #target table feature engineering
    df_posts_test=post_table_feature_engineer(df_posts)

    #merge all tables
    df=big_merge(posts_share, posts_comment, posts_like, posts_collect, df_posts_test)

    #deal with categorical features
    df_test=create_dummy(df, ['created_weekday', 'created_time'])
    
    #load trained model
    model=load_model(args.model)
    
    #predict post
    predict=predict_model(model, df_test)
    
    #save output
    save_output(df_posts_test, predict, args.output)